In [ ]:
import pandas as pd
import subprocess

For this tutorial, we will be using samples published by [Wibowo et al. 2021](https://www.nature.com/articles/s41586-021-03532-0)

In [ ]:
metadata = pd.read_csv("../data/metadata/PRJNA561510.tsv", sep="\t")

We select only the Zape samples

In [ ]:
zape = metadata.query("scientific_name == 'human gut metagenome' and experiment_alias.str.contains('Zape')", engine='python')
zape

In [ ]:
def download(sample_name, url, outdir):
    fwd = url.split(";")[0]
    rev = url.split(";")[1]
    dl_fwd = f"wget -O {outdir}/{sample_name}_1.fastq.gz {fwd}"
    print(dl_fwd)
    subprocess.check_output(dl_fwd, shell=True)
    dl_rev = f"wget -O {outdir}/{sample_name}_2.fastq.gz {rev}"
    print(dl_rev)
    subprocess.check_output(dl_rev, shell=True)

In [ ]:
for i in zape.index:
    download(sample_name = zape.loc[i, 'library_name'],
             url = zape.loc[i, 'fastq_ftp'],
             outdir = "../data/raw")